In [12]:
!git clone https://github.com/a2aniket/small-LLM

Cloning into 'small-LLM'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), 9.61 KiB | 9.61 MiB/s, done.


In [13]:
from transformer import Transformer # this is the transformer.py file
import torch
import numpy as np

In [14]:
# Generated this by filtering Appendix code

START_TOKEN = '<START>'
PADDING_TOKEN = '<PAD>'
END_TOKEN = '<END>'
marathi_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',

                     '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ',

                     # Marathi Vowels (स्वर)
                     'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ॠ', 'ऌ', 'ॡ', 'ए', 'ऐ', 'ओ', 'औ',

                     # Marathi Consonants (व्यंजने)
                     'क', 'ख', 'ग', 'घ', 'ङ',
                     'च', 'छ', 'ज', 'झ', 'ञ',
                     'ट', 'ठ', 'ड', 'ढ', 'ण',
                     'त', 'थ', 'द', 'ध', 'न',
                     'प', 'फ', 'ब', 'भ', 'म',
                     'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 'ळ', 'क्ष', 'ज्ञ',

                     # Matras and Other Symbols
                     'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'ॢ', 'ॣ', 'े', 'ै', 'ो', 'ौ',
                     '्', 'ं', 'ः', '़', 'ॅ', 'ॉ',

                     # Marathi Numbers
                     '०', '१', '२', '३', '४', '५', '६', '७', '८', '९',

                     PADDING_TOKEN, END_TOKEN]


english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [15]:
index_to_marathi = {k:v for k,v in enumerate(marathi_vocabulary)}
marathi_to_index = {v:k for k,v in enumerate(marathi_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [16]:
english_sentences=[ "he lost his key", "he lost his phone", "he lost his laptop", "he lost his wallet", "he lost his bag", "she lost her key", "she lost her phone", "she lost her laptop", "she lost her wallet", "she lost her bag", "i lost my key", "i lost my phone", "i lost my laptop", "i lost my wallet", "i lost my bag", "they lost their key", "they lost their phone", "they lost their laptop", "they lost their wallet", "they lost their bag", "we found the key", "we found the phone", "we found the laptop", "we found the wallet", "we found the bag", "he found his key", "he found his phone", "he found his laptop", "he found his wallet", "he found his bag", "she found her key", "she found her phone", "she found her laptop", "she found her wallet", "she found her bag", "i found my key", "i found my phone", "i found my laptop", "i found my wallet", "i found my bag", "they found their key", "they found their phone", "they found their laptop", "they found their wallet", "they found their bag", "the key is here", "the phone is here", "the laptop is here", "the wallet is here", "the bag is here" ]

In [17]:
marathi_sentences = [ "त्याने त्याची किल्ली हरवली", "त्याने त्याचा फोन हरवला", "त्याने त्याचा लॅपटॉप हरवला", "त्याने त्याचे पाकीट हरवले", "त्याने त्याची बॅग हरवली", "तिने तिची किल्ली हरवली", "तिने तिचा फोन हरवला", "तिने तिचा लॅपटॉप हरवला", "तिने तिचे पाकीट हरवले", "तिने तिची बॅग हरवली", "मी माझी किल्ली हरवली", "मी माझा फोन हरवला", "मी माझा लॅपटॉप हरवला", "मी माझे पाकीट हरवले", "मी माझी बॅग हरवली", "त्यांनी त्यांची किल्ली हरवली", "त्यांनी त्यांचा फोन हरवला", "त्यांनी त्यांचा लॅपटॉप हरवला", "त्यांनी त्यांचे पाकीट हरवले", "त्यांनी त्यांची बॅग हरवली", "आम्हाला किल्ली सापडली", "आम्हाला फोन सापडला", "आम्हाला लॅपटॉप सापडला", "आम्हाला पाकीट सापडले", "आम्हाला बॅग सापडली", "त्याला त्याची किल्ली सापडली", "त्याला त्याचा फोन सापडला", "त्याला त्याचा लॅपटॉप सापडला", "त्याला त्याचे पाकीट सापडले", "त्याला त्याची बॅग सापडली", "तिला तिची किल्ली सापडली", "तिला तिचा फोन सापडला", "तिला तिचा लॅपटॉप सापडला", "तिला तिचे पाकीट सापडले", "तिला तिची बॅग सापडली", "मला माझी किल्ली सापडली", "मला माझा फोन सापडला", "मला माझा लॅपटॉप सापडला", "मला माझे पाकीट सापडले", "मला माझी बॅग सापडली", "त्यांना त्यांची किल्ली सापडली", "त्यांना त्यांचा फोन सापडला", "त्यांना त्यांचा लॅपटॉप सापडला", "त्यांना त्यांचे पाकीट सापडले", "त्यांना त्यांची बॅग सापडली", "किल्ली इथे आहे", "फोन इथे आहे", "लॅपटॉप इथे आहे", "पाकीट इथे आहे", "बॅग इथे आहे" ]

In [18]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Kannada: {np.percentile([len(x) for x in marathi_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )

97th percentile length Kannada: 28.53
97th percentile length English: 22.53


In [19]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(marathi_sentences)):
    marathi_sentence, english_sentence = marathi_sentences[index], english_sentences[index]
    if is_valid_length(marathi_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(marathi_sentence, marathi_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(marathi_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 50
Number of valid sentences: 50


In [20]:
marathi_sentences = [marathi_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [21]:
marathi_sentences[:3]

['त्याने त्याची किल्ली हरवली',
 'त्याने त्याचा फोन हरवला',
 'त्याने त्याचा लॅपटॉप हरवला']

In [22]:
import torch

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
m_vocab_size = len(marathi_vocabulary)

transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          m_vocab_size,
                          english_to_index,
                          marathi_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [23]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, marathi_sentences):
        self.english_sentences = english_sentences
        self.marathi_sentences = marathi_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.marathi_sentences[idx]

In [24]:
dataset = TextDataset(english_sentences, marathi_sentences)


In [25]:
len(dataset)
dataset[1]


('he lost his phone', 'त्याने त्याचा फोन हरवला')

In [26]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [27]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('he lost his key', 'he lost his phone', 'he lost his laptop', 'he lost his wallet', 'he lost his bag', 'she lost her key', 'she lost her phone', 'she lost her laptop', 'she lost her wallet', 'she lost her bag', 'i lost my key', 'i lost my phone', 'i lost my laptop', 'i lost my wallet', 'i lost my bag', 'they lost their key', 'they lost their phone', 'they lost their laptop', 'they lost their wallet', 'they lost their bag', 'we found the key', 'we found the phone', 'we found the laptop', 'we found the wallet', 'we found the bag', 'he found his key', 'he found his phone', 'he found his laptop', 'he found his wallet', 'he found his bag'), ('त्याने त्याची किल्ली हरवली', 'त्याने त्याचा फोन हरवला', 'त्याने त्याचा लॅपटॉप हरवला', 'त्याने त्याचे पाकीट हरवले', 'त्याने त्याची बॅग हरवली', 'तिने तिची किल्ली हरवली', 'तिने तिचा फोन हरवला', 'तिने तिचा लॅपटॉप हरवला', 'तिने तिचे पाकीट हरवले', 'तिने तिची बॅग हरवली', 'मी माझी किल्ली हरवली', 'मी माझा फोन हरवला', 'मी माझा लॅपटॉप हरवला', 'मी माझे पाकीट हरव

In [28]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=marathi_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [29]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, m_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      kn_chars_to_padding_mask = np.arange(m_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [30]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 250

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, kn_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, kn_batch)
        optim.zero_grad()
        kn_predictions = transformer(eng_batch,
                                     kn_batch,
                                     encoder_self_attention_mask.to(device),
                                     decoder_self_attention_mask.to(device),
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(kn_batch, start_token=False, end_token=True)
        loss = criterian(
            kn_predictions.view(-1,m_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == marathi_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Marathi Translation: {kn_batch[0]}")
            m_sentence_predicted = torch.argmax(kn_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in m_sentence_predicted:
              if idx == marathi_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_marathi[idx.item()]
            print(f"Marathi Prediction: {predicted_sentence}")


            transformer.eval()
            m_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, m_sentence)
                predictions = transformer(eng_sentence,
                                          m_sentence,
                                          encoder_self_attention_mask.to(device),
                                          decoder_self_attention_mask.to(device),
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_marathi[next_token_index]
                m_sentence = (m_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break

            print(f"Evaluation translation (should we go to the mall?) : {m_sentence}")
            print("-------------------------------------------")

Epoch 0
Iteration 0 : 5.641683101654053
English: he lost his key
Marathi Translation: त्याने त्याची किल्ली हरवली
Marathi Prediction: २ळअळळ<ळळळअळअअअअळळळळअळळ ळळळळआॅॅडब9ढ८णक्षूछडणॅउक्षब७5छआछआपफआछछक्ष७$ड७छछछफछछछछड6णअछफघछछछछछछछ७छछछछछछछछछछछछछघछछछ?छछ७ृछछअछछछछछछछछणछछछछछछछछछछछछछछछ७छक्षऌ७छडडडड6छ२*अछक्ष!छछछछछ*1फछफफछलडलडडफडडफ*अफघफफफछ?छउणऌऌछछअ66क्षडडछछळदअ1
Evaluation translation (should we go to the mall?) : ('ल  ल        लल              ललललललल      लल    ललललललललललललललल           ललललललललललललललललललललल ललललललललललललललललललललललललललललललललललललललललललललललललललल   ललललललललललललललललललललललललललललललललललललललललललललललडड   ',)
-------------------------------------------
Epoch 1
Iteration 0 : 4.011083126068115
English: he lost his key
Marathi Translation: त्याने त्याची किल्ली हरवली
Marathi Prediction: लापलाााााालाललललााााा  ललाललललललााालबक्षबौपाााालपललललललालपलललाालााछलछलापााछाललललललललललललललललललछलाछलपपललललललललललललललालललाललललछलल७ललललललललछलललललाक्षालललललल(छललछपपललापलालालललललललललाााललललललललललललललल?लललछ?ापा
Evaluation tr

In [31]:

transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  m_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, m_sentence)
    predictions = transformer(eng_sentence,
                              m_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_marathi[next_token_index]
    m_sentence = (m_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return m_sentence[0]

In [34]:
translation = translate("he lost his brain")
print(translation)

त्याने त्याचा फोन हरवली<END>
